## 1. Import packages

In [1]:
# Import packages
import hail as hl
from bokeh.io import output_notebook,show
import gnomad.utils.vep

## 2. Import data

In [2]:
# Import gnomaAD v.3.1.2
ht = hl.read_table('gs://gcp-public-data--gnomad/release/3.1.2/ht/genomes/gnomad.genomes.v3.1.2.sites.ht')
ht = ht.head(100000) # Subset the data

# Import mutation rates from gnomAD paper
ht_mu = hl.import_table('data/supplementary_dataset_10_mutation_rates.tsv.gz',
                delimiter='\t', impute=True, force_bgz=True)

# Import context table from gnomad (https://broadinstitute.github.io/gnomad_methods/api_reference/utils/vep.html?highlight=context#gnomad.utils.vep.get_vep_context)
context_table = gnomad.utils.vep.get_vep_context("GRCh38").ht()
context_table_parsed = context_table.select(context_table.context)
context_table_parsed = context_table_parsed.transmute(context = context_table_parsed.context[2:5])

Initializing Hail with default parameters...


2022-09-23 00:25:22 WARN  Utils:69 - Your hostname, MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.181 instead (on interface en0)
2022-09-23 00:25:22 WARN  Utils:69 - Set SPARK_LOCAL_IP if you need to bind to another address


2022-09-23 00:25:22 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.1.3
SparkUI available at http://192.168.0.181:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.99-57537fea08d4
LOGGING: writing to /Users/adrian/BroadIS/01_maps/hail-20220923-0025-0.2.99-57537fea08d4.log
2022-09-23 00:25:37 Hail: INFO: Reading table to impute column types
2022-09-23 00:25:39 Hail: INFO: Finished type imputation
  Loading field 'context' as type str (imputed)
  Loading field 'ref' as type str (imputed)
  Loading field 'alt' as type str (imputed)
  Loading field 'methylation_level' as type int32 (imputed)
  Loading field 'mu_snp' as type float64 (imputed)


### Show the data structure

In [3]:
ht.show(3)

+---------------+------------+
| locus         | alleles    |
+---------------+------------+
| locus<GRCh38> | array<str> |
+---------------+------------+
| chr1:10031    | ["T","C"]  |
| chr1:10037    | ["T","C"]  |
| chr1:10043    | ["T","C"]  |
+---------------+------------+

+------------------------------------------------------------------------------+
| freq                                                                         |
+------------------------------------------------------------------------------+
| array<struct{AC: int32, AF: float64, AN: int32, homozygote_count: int32}>    |
+------------------------------------------------------------------------------+
| [(0,0.00e+00,56642,0),(2,1.65e-05,121094,0),(0,0.00e+00,25546,0),(0,0.00e... |
| [(2,2.60e-05,76882,0),(4,3.15e-05,126902,0),(0,0.00e+00,34670,0),(1,1.80e... |
| [(1,1.17e-05,85634,0),(1,8.24e-06,121430,0),(0,0.00e+00,39236,0),(0,0.00e... |
+------------------------------------------------------------------------------+

+------------------------------------------------------------------------------+
| raw_qual_hists.gq_hist_all.bin_edges                                         |
+------------------------------------------------------------------------------+
| array<float64>                                                               |
+------------------------------------------------------------------------------+
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
+------------------------------------------------------------------------------+

+------------------------------------------------------------------------------+
| raw_qual_hists.gq_hist_all.bin_freq                                          |
+------------------------------------------------------------------------------+
| array<int64>                                                                 |
+------------------------------------------------------------------------------+
| [14149,7330,6840,3904,23217,2436,1647,594,176,118,49,29,19,9,11,6,5,2,4,2]   |
| [10121,5441,5610,3838,29394,3741,2868,1281,401,358,163,63,69,31,18,20,10,... |
| [6578,3782,4202,3336,31093,4532,3696,1752,595,585,261,88,82,59,17,26,14,5... |
+------------------------------------------------------------------------------+

+--------------------------------------+-------------------------------------+
| raw_qual_hists.gq_hist_all.n_smaller | raw_qual_hists.gq_hist_all.n_larger |
+--------------------------------------+-------------------------------------+
|                                int64 |                               int64 |
+--------------------------------------+-------------------------------------+
|                                    0 |                                   0 |
|                                    0 |                                   0 |
|                                    0 |                                   0 |
+--------------------------------------+-------------------------------------+

+------------------------------------------------------------------------------+
| raw_qual_hists.dp_hist_all.bin_edges                                         |
+------------------------------------------------------------------------------+
| array<float64>                                                               |
+------------------------------------------------------------------------------+
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
+------------------------------------------------------------------------------+

+------------------------------------------------------------------------------+
| raw_q

In [4]:
# Table with methylation level and mutational rate in the trinucleotide context
ht_mu.show(3)

,,,,
context,ref,alt,methylation_level,mu_snp
str,str,str,int32,float64
"""AAA""","""A""","""C""",0,1.83e-09
"""AAA""","""A""","""G""",0,3.03e-09
"""AAA""","""A""","""T""",0,1.07e-09


In [5]:
# This table contains already precalculated nucleotides -3/+3 from mutation site 
context_table_parsed.show(3)

,,
locus,alleles,context
locus<GRCh38>,array<str>,str
chr1:10001,"[""T"",""A""]","""NTA"""
chr1:10001,"[""T"",""C""]","""NTA"""
chr1:10001,"[""T"",""G""]","""NTA"""


## 3. Add context field to main data

In [6]:
# Before joining the tri-nucleotide context of mutation
ht.count()

100000

In [7]:
# Join only matching rows from context to ht table.
ht = ht.key_by('locus', 'alleles').join(context_table_parsed.key_by('locus', 'alleles'), how = 'left')

In [8]:
# After
ht.count()

100000

## 4. Add mutation rates for added contexts

In [9]:
# Split alleles field to ref and alt allele
ht = ht.annotate(ref=ht.alleles[0], alt=ht.alleles[1])

# Add mutation rates according to the context, but also ref and alt allele for this context
ht = ht.key_by("context", "ref", "alt").join(ht_mu.key_by("context", "ref", "alt"), how = 'left')

In [10]:
# After adding context and mutation rates to the main table 
# (can be more than original number of rows as context may occure more than once depending on the locus)
ht.count()

2022-09-23 00:27:24 Hail: INFO: Ordering unsorted dataset with network shuffle4]
2022-09-23 00:27:25 Hail: INFO: Coerced sorted dataset


105026

In [11]:
# Add merged context with ref and alt variants for later table with mutation rates
ht = ht.annotate(context_ref_alt = ht.context + '_' + ht.ref + '_' + ht.alt)

# Change key for grouping to merged context
ht = ht.key_by('context_ref_alt')

In [12]:
# Show that contexts may be the same, but locus is completely different
ht.show(20)

2022-09-23 00:28:50 Hail: INFO: Ordering unsorted dataset with network shuffle4]
2022-09-23 00:28:51 Hail: INFO: Coerced sorted dataset


+---------+-----+-----+---------------+------------+
| context | ref | alt | locus         | alleles    |
+---------+-----+-----+---------------+------------+
| str     | str | str | locus<GRCh38> | array<str> |
+---------+-----+-----+---------------+------------+
| "AAA"   | "A" | "C" | chr1:10256    | ["A","C"]  |
| "AAA"   | "A" | "C" | chr1:13673    | ["A","C"]  |
| "AAA"   | "A" | "C" | chr1:13869    | ["A","C"]  |
| "AAA"   | "A" | "C" | chr1:14194    | ["A","C"]  |
| "AAA"   | "A" | "C" | chr1:14279    | ["A","C"]  |
| "AAA"   | "A" | "C" | chr1:14911    | ["A","C"]  |
| "AAA"   | "A" | "C" | chr1:14912    | ["A","C"]  |
| "AAA"   | "A" | "C" | chr1:15050    | ["A","C"]  |
| "AAA"   | "A" | "C" | chr1:15282    | ["A","C"]  |
| "AAA"   | "A" | "C" | chr1:16315    | ["A","C"]  |
| "AAA"   | "A" | "C" | chr1:16322    | ["A","C"]  |
| "AAA"   | "A" | "C" | chr1:16509    | ["A","C"]  |
| "AAA"   | "A" | "C" | chr1:19145    | ["A","C"]  |
| "AAA"   | "A" | "C" | chr1:19599    | ["A","C"]  |
| "AAA"   | "A" | "C" | chr1:19988    | ["A","C"]  |
| "AAA"   | "A" | "C" | chr1:20096    | ["A","C"]  |
| "AAA"   | "A" | "C" | chr1:20554    | ["A","C"]  |
| "AAA"   | "A" | "C" | chr1:20582    | ["A","C"]  |
| "AAA"   | "A" | "C" | chr1:23200    | ["A","C"]  |
| "AAA"   | "A" | "C" | chr1:30258    | ["A","C"]  |
+---------+-----+-----+---------------+------------+

+------------------------------------------------------------------------------+
| freq                                                                         |
+------------------------------------------------------------------------------+
| array<struct{AC: int32, AF: float64, AN: int32, homozygote_count: int32}>    |
+------------------------------------------------------------------------------+
| [(0,0.00e+00,82,0),(1,2.03e-03,492,0),(0,0.00e+00,40,0),(0,0.00e+00,2,0),... |
| [(1,7.13e-06,140208,0),(4,2.65e-05,150678,0),(0,0.00e+00,60710,0),(0,0.00... |
| [(0,0.00e+00,108984,0),(1,6.80e-06,147046,0),(0,0.00e+00,42554,0),(0,0.00... |
| [(0,0.00e+00,89562,0),(1,6.96e-06,143634,0),(0,0.00e+00,33976,0),(0,0.00e... |
| [(1,1.10e-05,91018,0),(3,2.10e-05,142760,0),(0,0.00e+00,36046,0),(0,0.00e... |
| [(2,1.51e-05,132814,0),(4,2.66e-05,150552,0),(0,0.00e+00,58664,0),(0,0.00... |
| [(1,7.55e-06,132418,0),(4,2.66e-05,150338,0),(0,0.00e+00,58498,0),(0,0.00... |
| [(0,0.00e+00,91994,0),(3,2.10e-05,143164,0),(0,0.00e+00,39696,0),(0,0.00e... |
| [(0,0.00e+00,63796,0),(2,1.48e-05,135462,1),(0,0.00e+00,26866,0),(0,0.00e... |
| [(2,1.33e-05,150484,0),(11,7.23e-05,152088,0),(2,2.97e-05,67348,0),(0,0.0... |
| [(1,6.65e-06,150472,0),(2,1.32e-05,152076,0),(1,1.48e-05,67348,0),(0,0.00... |
| [(2,1.31e-05,152276,0),(5,3.28e-05,152308,0),(2,2.94e-05,68036,0),(0,0.00... |
| [(5,4.27e-05,116966,0),(20,1.37e-04,145854,0),(1,1.95e-05,51166,0),(0,0.0... |
| [(0,0.00e+00,66928,0),(2,1.56e-05,128466,0),(0,0.00e+00,26386,0),(0,0.00e... |
| [(2,3.11e-05,64334,0),(2,1.52e-05,131912,0),(0,0.00e+00,23688,0),(0,0.00e... |
| [(52,6.82e-04,76258,0),(218,2.33e-03,93494,2),(0,0.00e+00,23808,0),(0,0.0... |
| [(2,1.98e-05,101254,0),(8,5.51e-05,145170,0),(0,0.00e+00,43812,0),(0,0.00... |
| [(0,0.00e+00,73488,0),(1,7.40e-06,135190,0),(0,0.00e+00,30058,0),(0,0.00e... |
| [(705,4.97e-02,14176,3),(3018,3.54e-02,85178,344),(5,1.58e-03,3164,1),(0,... |
| [(0,NA,0,0),(2,1.45e-04,13754,1),(0,NA,0,0),(0,NA,0,0),(0,NA,0,0),(0,NA,0... |
+------------------------------------------------------------------------------+

+------------------------------------------------------------------------------+
| raw_qual_hists.gq_hist_all.bin_edges                                         |
+------------------------------------------------------------------------------+
| array<float64>                                                               |
+------------------------------------------------------------------------------+
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
| [0.00e+00,5.00e+00,1.00e+0

#### *From this point the main key to group tables is by `context_ref_alt`*

## 5. Train linear model on synonymous variants for mutational class correction

In [13]:
# Filter only synonymous variants
ht_syn = ht.filter(ht.vep.most_severe_consequence == "synonymous_variant")

# Calculate number of variants in each tri-nucleotide context in synonymous variants
ht_syn_N_variants = (ht_syn.group_by(ht_syn.context_ref_alt).aggregate(N_variants = hl.agg.count()))

# Calculate number of singletons for each tri-nucleotide context in synonymous variants
ht_syn_singletons = ht_syn.filter(ht_syn.info.singleton == 1)
ht_syn_N_singletons = (ht_syn_singletons.group_by(ht_syn_singletons.context_ref_alt).aggregate(N_singletons = hl.agg.count()))

# Merge the N variants and N singletons tables
ht_syn_ps  = ht_syn_N_variants.join(ht_syn_N_singletons, how = 'outer') # outer as all will match and we want all
ht_syn_ps = ht_syn_ps.annotate(ps = ht_syn_ps.N_singletons/ht_syn_ps.N_variants)

2022-09-23 00:29:40 Hail: INFO: Table.join: renamed the following fields on the right to avoid name conflicts:
    'faf_meta' -> 'faf_meta_1'
    'vep_help' -> 'vep_help_1'
    'freq_index_dict' -> 'freq_index_dict_1'
    'freq_sample_count' -> 'freq_sample_count_1'
    'faf_index_dict' -> 'faf_index_dict_1'
    'grange' -> 'grange_1'
    'freq_meta' -> 'freq_meta_1'
    'dbsnp_version' -> 'dbsnp_version_1'
    'vep_version' -> 'vep_version_1'
    'age_distribution' -> 'age_distribution_1'
    'vep_csq_header' -> 'vep_csq_header_1'
    'vep_config' -> 'vep_config_1'
    'filtering_model' -> 'filtering_model_1'


### Show input table for regression

In [14]:
# How many rows before adding mutation rates
ht_syn_ps.count()

2022-09-23 00:30:22 Hail: INFO: Ordering unsorted dataset with network shuffle4]
2022-09-23 00:30:22 Hail: INFO: Coerced sorted dataset
2022-09-23 00:31:03 Hail: INFO: Coerced sorted dataset            (10 + 4) / 14]
2022-09-23 00:31:46 Hail: INFO: Ordering unsorted dataset with network shuffle4]
2022-09-23 00:31:47 Hail: INFO: Coerced sorted dataset
2022-09-23 00:32:30 Hail: INFO: Coerced sorted dataset             (8 + 6) / 14]


69

In [15]:
# Add mutation rate back to the table
#ht_syn_ps = ht_syn_ps.join(ht.select(ht.mu_snp), how = 'left')
# With the code below it doesn't keep duplicates, which is good as main table has contexts annotated to more
# than one variant, causing the duplication as the mu_snp is sometimes more than once appearing in the table)
ht_syn_ps = ht_syn_ps.annotate(**ht.select(ht.mu_snp)[ht_syn_ps.context_ref_alt])

#ht1.annotate(**ht2[ht1.x1])

ht_syn_ps.show(3)

2022-09-23 00:33:13 Hail: INFO: Ordering unsorted dataset with network shuffle4]
2022-09-23 00:33:13 Hail: INFO: Coerced sorted dataset
2022-09-23 00:33:55 Hail: INFO: Coerced sorted dataset             (8 + 6) / 14]
2022-09-23 00:34:36 Hail: INFO: Ordering unsorted dataset with network shuffle4]
2022-09-23 00:34:37 Hail: INFO: Coerced sorted dataset
2022-09-23 00:35:17 Hail: INFO: Coerced sorted dataset             (8 + 6) / 14]
2022-09-23 00:35:58 Hail: INFO: Ordering unsorted dataset with network shuffle4]
2022-09-23 00:35:58 Hail: INFO: Coerced sorted dataset
2022-09-23 00:36:37 Hail: INFO: Coerced sorted dataset>           (11 + 3) / 14]


,,,,
context_ref_alt,N_variants,N_singletons,ps,mu_snp
str,int64,int64,float64,float64
"""AAT_A_G""",1,1,1.00e+00,7.75e-09
"""ACA_C_T""",2,NA,NA,9.32e-09
"""ACC_C_A""",1,NA,NA,5.70e-09


In [16]:
# How many rows after adding mutation rates
ht_syn_ps.count()

2022-09-23 00:37:21 Hail: INFO: Ordering unsorted dataset with network shuffle4]
2022-09-23 00:37:22 Hail: INFO: Coerced sorted dataset
2022-09-23 00:38:02 Hail: INFO: Coerced sorted dataset============(14 + 0) / 14]
2022-09-23 00:38:48 Hail: INFO: Ordering unsorted dataset with network shuffle4]
2022-09-23 00:38:48 Hail: INFO: Coerced sorted dataset
2022-09-23 00:39:29 Hail: INFO: Coerced sorted dataset====>       (12 + 2) / 14]


69

### Perform regression

In [17]:
# Perform regression
ht_syn_lm = ht_syn_ps.aggregate(hl.agg.linreg(ht_syn_ps.ps, [1, ht_syn_ps.mu_snp], weight=ht_syn_ps.N_variants).beta)

# Show intercept and beta
ht_syn_lm

2022-09-23 00:40:14 Hail: INFO: Ordering unsorted dataset with network shuffle4]
2022-09-23 00:40:14 Hail: INFO: Coerced sorted dataset
2022-09-23 00:40:55 Hail: INFO: Coerced sorted dataset             (8 + 6) / 14]
2022-09-23 00:41:35 Hail: INFO: Ordering unsorted dataset with network shuffle4]
2022-09-23 00:41:36 Hail: INFO: Coerced sorted dataset
2022-09-23 00:42:16 Hail: INFO: Coerced sorted dataset             (8 + 6) / 14]
2022-09-23 00:42:56 Hail: INFO: Ordering unsorted dataset with network shuffle4]
2022-09-23 00:42:57 Hail: INFO: Coerced sorted dataset
2022-09-23 00:43:37 Hail: INFO: Coerced sorted dataset             (8 + 6) / 14]


[0.6200349002747344, 11991113.311757443]

## 6. Predict expected number of variants for each context

### For testing purposes focus on `3_prime_UTR_variant`

### Function for regression eventually will be made starting here and put in `/utils/utils.py` script

In [18]:
# Filter specific variant functionall class for calculating MAPS in
ht_reg_table = ht.filter(ht.vep.most_severe_consequence == "3_prime_UTR_variant")

# Count number of variants and singletons
ht_reg_table_N_variants = (ht_reg_table.group_by(ht_reg_table.context_ref_alt).aggregate(N_variants = hl.agg.count()))
ht_reg_table_N_singletons = (ht_reg_table.group_by(ht_reg_table.context_ref_alt).aggregate(N_singletons = hl.agg.sum(ht_reg_table.info.singleton)))

# Merge the tables to obtain proportions (ps)
ht_reg_table_ps = ht_reg_table_N_variants.join(ht_reg_table_N_singletons, how = "outer") # outer as we want all anyway
ht_reg_table_ps = ht_reg_table_ps.annotate(ps = ht_reg_table_ps.N_singletons/ht_reg_table_ps.N_variants)

# Add mutation rate back to the table (mu_snp matching key from variants table)
#ht_reg_table_ps = ht_reg_table_ps.join(ht.select(ht.mu_snp), how = 'left')
# With the code below it doesn't keep duplicates, which is good as main table has contexts annotated to more
# than one variant, causing the duplication as the mu_snp is sometimes more than once appearing in the table)
ht_reg_table_ps = ht_reg_table_ps.annotate(**ht.select(ht.mu_snp)[ht_reg_table_ps.context_ref_alt])

# Get expected number of singletons by applying the model factors
ht_reg_table_ps_lm = ht_reg_table_ps.annotate(expected_singletons=(ht_reg_table_ps.mu_snp * ht_syn_lm[1] + ht_syn_lm[0]) * ht_reg_table_ps.N_variants)

# Now aggregate into the main consequence from contexts
ht_reg_table_ps_lm_cons = ht_reg_table_ps_lm.annotate(consequence = "3_prime_UTR_variant")

# To aggregate just sum for the context
ht_reg_table_ps_lm_cons_agg = (ht_reg_table_ps_lm_cons.group_by("consequence")
              .aggregate(N_singletons=hl.agg.sum(ht_reg_table_ps_lm_cons.N_singletons),
                         expected_singletons=hl.agg.sum(ht_reg_table_ps_lm_cons.expected_singletons),
                         N_variants=hl.agg.sum(ht_reg_table_ps_lm_cons.N_variants)))

# Calculate MAPS and aggregated proportions 
ht_reg_table_ps_lm_cons_agg_MAPS = ht_reg_table_ps_lm_cons_agg.annotate(ps_agg=ht_reg_table_ps_lm_cons_agg.N_singletons / ht_reg_table_ps_lm_cons_agg.N_variants,
    maps=(ht_reg_table_ps_lm_cons_agg.N_singletons - ht_reg_table_ps_lm_cons_agg.expected_singletons) / ht_reg_table_ps_lm_cons_agg.N_variants)

# Add MAPS standard error of the mean (sem)
ht_reg_table_ps_lm_cons_agg_MAPS = ht_reg_table_ps_lm_cons_agg_MAPS.annotate(maps_sem=(ht_reg_table_ps_lm_cons_agg_MAPS.ps_agg * (1 - ht_reg_table_ps_lm_cons_agg_MAPS.ps_agg) / ht_reg_table_ps_lm_cons_agg_MAPS.N_variants) ** 0.5)


2022-09-23 00:43:42 Hail: INFO: Table.join: renamed the following fields on the right to avoid name conflicts:
    'faf_meta' -> 'faf_meta_1'
    'vep_help' -> 'vep_help_1'
    'freq_index_dict' -> 'freq_index_dict_1'
    'freq_sample_count' -> 'freq_sample_count_1'
    'faf_index_dict' -> 'faf_index_dict_1'
    'grange' -> 'grange_1'
    'freq_meta' -> 'freq_meta_1'
    'dbsnp_version' -> 'dbsnp_version_1'
    'vep_version' -> 'vep_version_1'
    'age_distribution' -> 'age_distribution_1'
    'vep_csq_header' -> 'vep_csq_header_1'
    'vep_config' -> 'vep_config_1'
    'filtering_model' -> 'filtering_model_1'


In [19]:
# Show final result
ht_reg_table_ps_lm_cons_agg_MAPS.show(20)

2022-09-23 00:44:25 Hail: INFO: Ordering unsorted dataset with network shuffle4]
2022-09-23 00:44:25 Hail: INFO: Coerced sorted dataset
2022-09-23 00:45:06 Hail: INFO: Coerced sorted dataset             (8 + 6) / 14]
2022-09-23 00:45:47 Hail: INFO: Ordering unsorted dataset with network shuffle4]
2022-09-23 00:45:47 Hail: INFO: Coerced sorted dataset
2022-09-23 00:46:27 Hail: INFO: Coerced sorted dataset             (9 + 5) / 14]
2022-09-23 00:47:07 Hail: INFO: Ordering unsorted dataset with network shuffle4]
2022-09-23 00:47:07 Hail: INFO: Coerced sorted dataset
2022-09-23 00:47:47 Hail: INFO: Coerced sorted dataset             (8 + 6) / 14]
2022-09-23 00:47:50 Hail: INFO: Coerced sorted dataset             (8 + 5) / 13]


,,,,,,
consequence,N_singletons,expected_singletons,N_variants,ps_agg,maps,maps_sem
str,int64,float64,int64,float64,float64,float64
"""3_prime_UTR_variant""",196,2.34e+02,720,2.72e-01,-5.35e-02,1.66e-02


In [22]:
## Define regression function
def regress_per_context(ht, cons):
    """
    Calculates MAPS with Standard Error of the Mean for a given consequence.
    
    Requires input in form of Hail table: (so the things that should be changes for easier accessibility):
        1) With context, ref allele, alt allele joined together as `context_ref_alt`,
        2) Mutation rates annotated for each of the `context_ref_alt`. 
    """
    # Filter specific variant functionall class for calculating MAPS in
    ht_reg_table = ht.filter(ht.vep.most_severe_consequence == cons) # cons = i.e 3_prime_UTR_variant

    # Count number of variants and singletons
    ht_reg_table_N_variants = (ht_reg_table.group_by(ht_reg_table.context_ref_alt).aggregate(N_variants = hl.agg.count()))
    ht_reg_table_N_singletons = (ht_reg_table.group_by(ht_reg_table.context_ref_alt).aggregate(N_singletons = hl.agg.sum(ht_reg_table.info.singleton)))

    # Merge the tables to obtain proportions (ps)
    ht_reg_table_ps = ht_reg_table_N_variants.join(ht_reg_table_N_singletons, how = "outer") # outer as we want all anyway
    ht_reg_table_ps = ht_reg_table_ps.annotate(ps = ht_reg_table_ps.N_singletons/ht_reg_table_ps.N_variants)

    # Add mutation rate back to the table (mu_snp matching key from variants table)
    #ht_reg_table_ps = ht_reg_table_ps.join(ht.select(ht.mu_snp), how = 'left')
    # With the code below it doesn't keep duplicates, which is good as main table has contexts annotated to more
    # than one variant, causing the duplication as the mu_snp is sometimes more than once appearing in the table)
    ht_reg_table_ps = ht_reg_table_ps.annotate(**ht.select(ht.mu_snp)[ht_reg_table_ps.context_ref_alt])

    # Get expected number of singletons by applying the model factors
    ht_reg_table_ps_lm = ht_reg_table_ps.annotate(expected_singletons=(ht_reg_table_ps.mu_snp * ht_syn_lm[1] + ht_syn_lm[0]) * ht_reg_table_ps.N_variants)

    # Now aggregate into the main consequence from contexts
    ht_reg_table_ps_lm_cons = ht_reg_table_ps_lm.annotate(consequence = cons) # cons = i.e 3_prime_UTR_variant

    # To aggregate just sum for the context
    ht_reg_table_ps_lm_cons_agg = (ht_reg_table_ps_lm_cons.group_by("consequence")
                  .aggregate(N_singletons=hl.agg.sum(ht_reg_table_ps_lm_cons.N_singletons),
                             expected_singletons=hl.agg.sum(ht_reg_table_ps_lm_cons.expected_singletons),
                             N_variants=hl.agg.sum(ht_reg_table_ps_lm_cons.N_variants)))

    # Calculate MAPS and aggregated proportions 
    ht_reg_table_ps_lm_cons_agg_MAPS = ht_reg_table_ps_lm_cons_agg.annotate(ps_agg=ht_reg_table_ps_lm_cons_agg.N_singletons / ht_reg_table_ps_lm_cons_agg.N_variants,
        maps=(ht_reg_table_ps_lm_cons_agg.N_singletons - ht_reg_table_ps_lm_cons_agg.expected_singletons) / ht_reg_table_ps_lm_cons_agg.N_variants)

    # Add MAPS standard error of the mean (sem)
    ht_reg_table_ps_lm_cons_agg_MAPS = ht_reg_table_ps_lm_cons_agg_MAPS.annotate(maps_sem=(ht_reg_table_ps_lm_cons_agg_MAPS.ps_agg * (1 - ht_reg_table_ps_lm_cons_agg_MAPS.ps_agg) / ht_reg_table_ps_lm_cons_agg_MAPS.N_variants) ** 0.5)
    return ht_reg_table_ps_lm_cons_agg_MAPS


In [25]:
regress_per_context(ht, '5_prime_UTR_variant'). show()

2022-09-23 13:15:59 Hail: INFO: Table.join: renamed the following fields on the right to avoid name conflicts:
    'faf_meta' -> 'faf_meta_1'
    'vep_help' -> 'vep_help_1'
    'freq_index_dict' -> 'freq_index_dict_1'
    'freq_sample_count' -> 'freq_sample_count_1'
    'faf_index_dict' -> 'faf_index_dict_1'
    'grange' -> 'grange_1'
    'freq_meta' -> 'freq_meta_1'
    'dbsnp_version' -> 'dbsnp_version_1'
    'vep_version' -> 'vep_version_1'
    'age_distribution' -> 'age_distribution_1'
    'vep_csq_header' -> 'vep_csq_header_1'
    'vep_config' -> 'vep_config_1'
    'filtering_model' -> 'filtering_model_1'
2022-09-23 13:17:00 Hail: INFO: Ordering unsorted dataset with network shuffle4]
2022-09-23 13:17:00 Hail: INFO: Coerced sorted dataset
2022-09-23 13:17:51 Hail: INFO: Coerced sorted dataset========>   (13 + 1) / 14]
2022-09-23 13:18:44 Hail: INFO: Ordering unsorted dataset with network shuffle4]
2022-09-23 13:18:45 Hail: INFO: Coerced sorted dataset
2022-09-23 13:19:34 Hail: INF

,,,,,,
consequence,N_singletons,expected_singletons,N_variants,ps_agg,maps,maps_sem
str,int64,float64,int64,float64,float64,float64
"""5_prime_UTR_variant""",1,7.18e-01,5,2.00e-01,5.65e-02,1.79e-01


In [ ]:
## Adapting per gene
def regress_per_context(ht, cons):
    """
    Calculates MAPS with Standard Error of the Mean for a given consequence.
    
    Requires input in form of Hail table: (so the things that should be changes for easier accessibility):
        1) With context, ref allele, alt allele joined together as `context_ref_alt`,
        2) Mutation rates annotated for each of the `context_ref_alt`. 
    """
    # Filter specific variant functionall class for calculating MAPS in
    ht_reg_table = ht.filter(ht.vep.most_severe_consequence == cons) # cons = i.e 3_prime_UTR_variant

    # Count number of variants and singletons
    ht_reg_table_N_variants = (ht_reg_table.group_by(ht_reg_table.context_ref_alt).aggregate(N_variants = hl.agg.count()))
    ht_reg_table_N_singletons = (ht_reg_table.group_by(ht_reg_table.context_ref_alt).aggregate(N_singletons = hl.agg.sum(ht_reg_table.info.singleton)))

    # Merge the tables to obtain proportions (ps)
    ht_reg_table_ps = ht_reg_table_N_variants.join(ht_reg_table_N_singletons, how = "outer") # outer as we want all anyway
    ht_reg_table_ps = ht_reg_table_ps.annotate(ps = ht_reg_table_ps.N_singletons/ht_reg_table_ps.N_variants)

    # Add mutation rate back to the table (mu_snp matching key from variants table)
    #ht_reg_table_ps = ht_reg_table_ps.join(ht.select(ht.mu_snp), how = 'left')
    # With the code below it doesn't keep duplicates, which is good as main table has contexts annotated to more
    # than one variant, causing the duplication as the mu_snp is sometimes more than once appearing in the table)
    ht_reg_table_ps = ht_reg_table_ps.annotate(**ht.select(ht.mu_snp)[ht_reg_table_ps.context_ref_alt])

    # Get expected number of singletons by applying the model factors
    ht_reg_table_ps_lm = ht_reg_table_ps.annotate(expected_singletons=(ht_reg_table_ps.mu_snp * ht_syn_lm[1] + ht_syn_lm[0]) * ht_reg_table_ps.N_variants)

    # Now aggregate into the main consequence from contexts
    ht_reg_table_ps_lm_cons = ht_reg_table_ps_lm.annotate(consequence = cons) # cons = i.e 3_prime_UTR_variant

    # To aggregate just sum for the context
    ht_reg_table_ps_lm_cons_agg = (ht_reg_table_ps_lm_cons.group_by("consequence")
                  .aggregate(N_singletons=hl.agg.sum(ht_reg_table_ps_lm_cons.N_singletons),
                             expected_singletons=hl.agg.sum(ht_reg_table_ps_lm_cons.expected_singletons),
                             N_variants=hl.agg.sum(ht_reg_table_ps_lm_cons.N_variants)))

    # Calculate MAPS and aggregated proportions 
    ht_reg_table_ps_lm_cons_agg_MAPS = ht_reg_table_ps_lm_cons_agg.annotate(ps_agg=ht_reg_table_ps_lm_cons_agg.N_singletons / ht_reg_table_ps_lm_cons_agg.N_variants,
        maps=(ht_reg_table_ps_lm_cons_agg.N_singletons - ht_reg_table_ps_lm_cons_agg.expected_singletons) / ht_reg_table_ps_lm_cons_agg.N_variants)

    # Add MAPS standard error of the mean (sem)
    ht_reg_table_ps_lm_cons_agg_MAPS = ht_reg_table_ps_lm_cons_agg_MAPS.annotate(maps_sem=(ht_reg_table_ps_lm_cons_agg_MAPS.ps_agg * (1 - ht_reg_table_ps_lm_cons_agg_MAPS.ps_agg) / ht_reg_table_ps_lm_cons_agg_MAPS.N_variants) ** 0.5)
    return ht_reg_table_ps_lm_cons_agg_MAPS
